## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing-deep")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = True
PLAY_RESULT = False
CLEARANCE = 1e-3
TOOL_RPY = (-np.pi/2,0,0)
TIMEOUT_MOTION = 5
MAX_TIME = 100


ROBOT_TYPE = RobotType.indy7gripper

if ROBOT_TYPE in [RobotType.indy7, RobotType.indy7gripper]:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.14)
    HOME_POSE = (0,0,0,0,0,0)
    GRIP_DEPTH = 0.05
    
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.112)
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
    GRIP_DEPTH = 0.03

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
indy0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## ui

In [11]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Object Classes

In [12]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask
from pkg.planning.filtering.lattice_model.scene_building import *

##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,1)
    
    def __init__(self, gscene, name, floor_height=None, *args, **kwargs):
        assert floor_height is not None, "floor_height needed"
        if floor_height > self.RTH_MIN[2]:
            self.RTH_MIN = self.RTH_MIN[:2]+(floor_height,)
        self.H = 0.4
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
            
    
##
# @class Box
# @brief box with the top and the front side open
class Box(WorkPlane):
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    COLOR =  (0.8,0.8,0.2,0.5)
    H_RANGE = (0.3, 0.6)
    THICKNESS = 0.05
    def __init__(self, gscene, name, H=None, **kwargs):
        WorkPlane.__init__(self, gscene=gscene, name=name, **kwargs)
        self.H = np.random.uniform(*self.H_RANGE) if H is None else H

        ## back wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_bw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(self.DIM[0]/2+self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## left wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_lw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, -self.DIM[1]/2-self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## right wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_rw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, self.DIM[1]/2+self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        
##
# @class SideBox
# @brief box with a side face open
class SideBox(Box):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, np.pi/6)
    RPY_MAX = (0, 0, np.pi/6)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    COLOR =  (0.2,0.2,0.8,1)
    H_RANGE = (0.4, 0.4)
    THICKNESS = 0.05
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## top
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_tp", link_name="base_link", 
            dims=(self.DIM[0], self.DIM[1], self.THICKNESS), center=(0, 0, self.H+self.THICKNESS/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

##
# @class TopBox
# @brief box with the top face open
class TopBox(Box):
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## front wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_fw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(-self.DIM[0]/2-self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))
        
##
# @class Floor
# @brief Floor - lowerbound of the workspace
class Floor(ObstacleBase):
    RTH_MIN = (0.0, 0, -0.5)
    RTH_MAX = (0.0, 0, -0.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    
##
# @class Pole
# @brief occasional poles that obstruct robot motion
class Pole(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.3, +np.pi, 0)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.1, 0.1, 4)
    DIM_MAX = (0.1, 0.1, 4)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi)
    RPY_MAX = (0, 0, +np.pi)
    DIM_MIN = (0.04, 0.1, 0.2)
    DIM_MAX = (0.04, 0.1, 0.2)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.8,0.2,1)
    def __init__(self, gscene, name, workplane, GRIP_DEPTH, CLEARANCE, XYZ_LOC=None, **kwargs):
        self.GRIP_DEPTH = GRIP_DEPTH
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.geometry.set_offset_tf(center = self.XYZ)
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
           
def disperse_objects(gscene, object_class, key, Nmax, workplane_on, GRIP_DEPTH, CLEARANCE=1e-3):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    while len(obs_list)<Nmax:
        iw = len(obs_list)
        obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)
        remove_this = False
        for obs_pre in obs_list:
            if obs_pre.is_overlapped_with(obs.geometry):
                remove_this = True
                break
        if remove_this:
            gscene.remove(obs.geometry)
        else:
            obs_list.append(obs)
    return obs_list

In [13]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep/indy7gripper/20210730-000918



## Init pybullet

In [14]:
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/developing/pddlstream'))
from convert_pscene import *
from plan_rnb import *
connect_notebook(use_gui=False)
urdf_pybullet_path = copy_meshes(gscene)

generate table - Geometry
generate table - Handle
generate table - Bindergenerate table - Object



### sampling 1-obj

In [16]:
Nmax_obj = 1
f_option = "obj_1"

SAMPLE_NUM = 100

SHOW_PERIOD = 0.01

GRASP_SAMPLE = 50

results_dict_1_shelf = defaultdict(list)
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = []
    for gtem in gscene:
        if gtem.link_name == "base_link":
            gtem_args.append(deepcopy(gtem.get_args()))
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))

============ STEP (0 / 100) / Elapsed: 0.1 s / ETA: 6.0 s =============
============ STEP (1 / 100) / Elapsed: 0.2 s / ETA: 10.8 s =============
============ STEP (2 / 100) / Elapsed: 0.3 s / ETA: 10.6 s =============
============ STEP (3 / 100) / Elapsed: 0.4 s / ETA: 10.4 s =============
============ STEP (4 / 100) / Elapsed: 0.5 s / ETA: 10.4 s =============
============ STEP (5 / 100) / Elapsed: 0.7 s / ETA: 11.2 s =============
============ STEP (6 / 100) / Elapsed: 0.9 s / ETA: 13.4 s =============
============ STEP (7 / 100) / Elapsed: 1.0 s / ETA: 13.0 s =============
============ STEP (8 / 100) / Elapsed: 1.1 s / ETA: 12.6 s =============
============ STEP (9 / 100) / Elapsed: 1.2 s / ETA: 12.4 s =============
============ STEP (10 / 100) / Elapsed: 1.4 s / ETA: 12.7 s =============
============ STEP (11 / 100) / Elapsed: 1.9 s / ETA: 15.6 s =============
============ STEP (12 / 100) / Elapsed: 2.0 s / ETA: 15.2 s =============
============ STEP (13 / 100) / Elapsed: 2.1 s / E

### sampling 3-obj

In [17]:
Nmax_obj = 3
f_option = "obj_3"

SAMPLE_NUM = 100

SHOW_PERIOD = 0.01

GRASP_SAMPLE = 50

results_dict_1_shelf = defaultdict(list)
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = []
    for gtem in gscene:
        if gtem.link_name == "base_link":
            gtem_args.append(deepcopy(gtem.get_args()))
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))

============ STEP (0 / 100) / Elapsed: 0.2 s / ETA: 15.7 s =============
============ STEP (1 / 100) / Elapsed: 0.3 s / ETA: 14.2 s =============
============ STEP (2 / 100) / Elapsed: 0.6 s / ETA: 19.1 s =============
============ STEP (3 / 100) / Elapsed: 0.8 s / ETA: 20.3 s =============
============ STEP (4 / 100) / Elapsed: 1.1 s / ETA: 22.0 s =============
============ STEP (5 / 100) / Elapsed: 1.3 s / ETA: 21.5 s =============
============ STEP (6 / 100) / Elapsed: 1.5 s / ETA: 21.7 s =============
============ STEP (7 / 100) / Elapsed: 1.7 s / ETA: 21.2 s =============
============ STEP (8 / 100) / Elapsed: 1.8 s / ETA: 20.2 s =============
============ STEP (9 / 100) / Elapsed: 2.0 s / ETA: 20.5 s =============
============ STEP (10 / 100) / Elapsed: 2.3 s / ETA: 21.2 s =============
============ STEP (11 / 100) / Elapsed: 2.5 s / ETA: 20.9 s =============
============ STEP (12 / 100) / Elapsed: 2.6 s / ETA: 20.3 s =============
============ STEP (13 / 100) / Elapsed: 2.8 s / 

## sample with pole

In [20]:
Nmax_obj = 3
f_option = "obj_3_pole"

SAMPLE_NUM = 100

SHOW_PERIOD = 0.01

GRASP_SAMPLE = 50

results_dict_1_shelf = defaultdict(list)
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}


gtimer = GlobalTimer.instance()
gtimer.reset(stack=True)
gtimer.tic("full_loop")
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)

#     pole = Pole(gscene, "pole", RTH=[0.3, gp.RTH[1], 0])

    Tgp = gp.geometry.get_tf(crob.home_pose)
    Pp = (np.matmul(Tgp[:3,:3], np.transpose([[-0.3,0,0]])*(1+np.random.rand()))+Tgp[:3,3:]).flatten()

    Ppcyl = cart2cyl(*Pp)
    pole = Pole(gscene, "pole", RTH=[np.maximum(0.3,Ppcyl[0]-0.1), Ppcyl[1], 0])


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

    gtem_args = []
    for gtem in gscene:
        if gtem.link_name == "base_link":
            gtem_args.append(deepcopy(gtem.get_args()))
    obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                for obj in obj_list}
    
    save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                {"gtem_args":gtem_args, "obj_args": obj_args})
    gscene.update_markers_all()
    print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                     *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))

============ STEP (0 / 100) / Elapsed: 0.1 s / ETA: 14.6 s =============
============ STEP (1 / 100) / Elapsed: 0.3 s / ETA: 14.0 s =============
============ STEP (2 / 100) / Elapsed: 0.7 s / ETA: 22.8 s =============
============ STEP (3 / 100) / Elapsed: 0.9 s / ETA: 23.1 s =============
============ STEP (4 / 100) / Elapsed: 1.1 s / ETA: 22.2 s =============
============ STEP (5 / 100) / Elapsed: 1.2 s / ETA: 20.7 s =============
============ STEP (6 / 100) / Elapsed: 1.4 s / ETA: 19.7 s =============
============ STEP (7 / 100) / Elapsed: 1.8 s / ETA: 22.9 s =============
============ STEP (8 / 100) / Elapsed: 2.3 s / ETA: 25.5 s =============
============ STEP (9 / 100) / Elapsed: 2.5 s / ETA: 25.3 s =============
============ STEP (10 / 100) / Elapsed: 2.7 s / ETA: 24.8 s =============
============ STEP (11 / 100) / Elapsed: 2.9 s / ETA: 24.3 s =============
============ STEP (12 / 100) / Elapsed: 3.0 s / ETA: 23.4 s =============
============ STEP (13 / 100) / Elapsed: 3.2 s / 